Authors: Sai Nudurupati & Erkan Istanbulluoglu

Date: 18 Apr 2017

This code tests the functioning of SourceTrackingAlgorithm.py, which is currently in the process of being inhaled as an utility of Landlab along with Ronda Strauch's updated LandslideProbability component which is being submitted to GMD shortly.

What does Source Tracking Algorithm (STA) do?
STA is a Landlab utility that operates on a grid, which is a Landlab’s RasterModelGrid instance, and records all upstream nodes for each node in the grid. 

Import required libraries

In [4]:
from landlab.io.esri_ascii import read_esri_ascii
from landlab.utils import source_tracking_algorithm as STA
import cPickle as pickle

In [5]:
import numpy as np
from landlab import RasterModelGrid as rmg
from landlab.components.flow_routing.route_flow_dn import FlowRouter

In this example, we will use a synthetic grid, which has two stream networks in itself. One of the stream network has multiple segments. This grid is assumed to present a comprehensive test of scenarios to legitimate the expected functioning of this utility.

In [6]:
## Eg5: 5X5 Elevation grid
grid = rmg(5,5,1.)
z = np.array([5., 5., 5., 5., 5.,
              5., 4., 5., 1., 5.,
              0., 3., 5., 3., 0.,
              5., 4., 5., 2., 5., 
              5., 5., 5., 5., 5.])

The input grid has two outlets. Since these outlets are perimeter nodes (inactive), these nodes will not be part of core_nodes. Hence, we need to change the status of these nodes to active.

In [7]:
grid.status_at_node[10] = 0
grid.status_at_node[14] = 0

You can see that these outlets are now 'core_nodes'.

In [8]:
grid.core_nodes

array([ 6,  7,  8, 10, 11, 12, 13, 14, 16, 17, 18])

Create a field 'topographic__elevation' on landlab grid to hold elevation data.

In [9]:
grid.at_node['topographic__elevation'] = z

Route flow using Landlab's component 'FlowRouter'. Landlab's FlowRouter creates a nodal field on the grid named 'flow__receiver_node' which points to the receiver node for each node in the grid. Check Landlab tutorials for using other Landlab tools to route flow across Digital Elevation Models (DEM) with flat surfaces.

In [10]:
fr = FlowRouter(grid)
fr.route_flow()
r = grid.at_node['flow__receiver_node']

In [11]:
r

array([ 0,  1,  2,  3,  4,  5, 10,  8, 14,  9, 10, 10,  8, 14, 14, 15, 10,
       18, 14, 19, 20, 21, 22, 23, 24])

In [12]:
r[18]

14

If one has ESRI ArcGIS derived flow directions, STA's method 'convert_arc_flow_directions_to_landlab_node_ids' to convert these flow directions into Landlab's receiver node id.

Sample code to use flow directions from ArcGIS:
grid, flow_dir_arc = read_esri_ascii('./Input_files/flow_direction.txt', name='flow_dir', grid=grid)
r = convert_arc_flow_directions_to_landlab_node_ids(grid, flow_dir_arc)

This code was originally written to derive weighted Recharge, from a coarser grid compared to the model domain, based on portions of upstream contributing area. Here 'hsd_ids' represent the ids of Hydrologic Source Domain (HSD). 

NOTE: If you would like to just derive all upstream ids of the model domain, provide ids of the model domain instead. Replacing the following cell with this code:
hsd_ids = grid.nodes.flatten()

In [13]:
hsd_ids = np.empty(grid.number_of_nodes, dtype=int)

For testing this algorithm, I am assuming that the grid nodes 2, 3, 4, 7, 8, 9 fall under HSD id '0' and rest of the grid nodes fall under HSD id '1'.

In [14]:
hsd_ids[:] = 1
hsd_ids[2:5] = 0
hsd_ids[7:10] = 0

Printing HSD ids

In [15]:
hsd_ids

array([1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1])

To record all upstream HSD ids, we will have to run the function 'track_source'.

In [16]:
(hsd_upstr, flow_accum) = STA.track_source(grid, r, hsd_ids)

hsd_upstr is a Python dictionary that maps every grid core node id to all contributing HSD ids {keys=model grid core node: values=upstream HSD ids contributing to this grid core node}

In [17]:
hsd_upstr

{6: [1],
 7: [0],
 8: [1, 0, 0],
 10: [1, 1, 1, 1],
 11: [1],
 12: [1],
 13: [1],
 14: [1, 1, 1, 1, 0, 0, 1],
 16: [1],
 17: [1],
 18: [1, 1]}

In [18]:
flow_accum

array([0, 0, 0, 0, 0, 0, 1, 1, 3, 0, 4, 1, 1, 1, 7, 0, 1, 1, 2, 0, 0, 0, 0,
       0, 0])

In [19]:
flow_accum[14]

7

If one wants to get the upstream HSD ids (without repitition) and fractions of contribution, run the function 'find_unique_upstream_hsd_ids_and_fractions()'

In [20]:
(uniq_ids, coeff) = STA.find_unique_upstream_hsd_ids_and_fractions(hsd_upstr)

'uniq_ids' is a Python dictionary that maps each grid core node id to all contributing HSD ids (without repitition) {keys=model grid core node: values=upstream HSD ids contributing to this grid core node without repitition}.

In [21]:
uniq_ids

{6: [1],
 7: [0],
 8: [0, 1],
 10: [1],
 11: [1],
 12: [1],
 13: [1],
 14: [0, 1],
 16: [1],
 17: [1],
 18: [1]}

'coeff' is a Python dictionary that maps each grid core node id to the fractions of contribution from HSD ids in the same order as uniq_ids[key] {keys=model grid core node: values=fractions of upstream HSD ids contributing to this node in the same order as uniq_ids[key]}

In [22]:
coeff

{6: [1.0],
 7: [1.0],
 8: [0.6666666666666666, 0.3333333333333333],
 10: [1.0],
 11: [1.0],
 12: [1.0],
 13: [1.0],
 14: [0.2857142857142857, 0.7142857142857143],
 16: [1.0],
 17: [1.0],
 18: [1.0]}